<a href="https://colab.research.google.com/github/lexvieira/Yandex_Google_Drive_Migration/blob/master/Yandex_Google_Drive_Migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
!pip install  yadisk
import yadisk
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import tempfile


In [ ]:
# Step 1: Connect to Yandex Disk (no token needed for public folders)
# y = yadisk.YaDisk()
token = 'y0__xCI9YP9BxjRwDcg6vyTgBNKIGsmtX0YDKv6buG8PZpQItKsvA'
y = yadisk.YaDisk(token="y0__xCI9YP9BxjRwDcg6vyTgBNKIGsmtX0YDKv6buG8PZpQItKsvA")
print(y.check_token()) # Check the token

True


In [ ]:
# Step 2: Authenticate Google Drive
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open("demofile.txt", "a") as f:
  f.write("Now the file has more content!")

#open and read the file after the appending:
with open("demofile.txt") as f:
  print(f.read())

Now the file has more content!


In [ ]:
# y.mkdir("/test") # Create folder
# y.upload("demofile.txt", "/test/demofile.txt") # Upload first file

<SyncResourceLinkObject{
    'href': 'https://cloud-api.yandex.net/v1/disk/resources?path=disk%3A%2Ftest%2Fdemofile.txt',
    'method': 'GET',
    'templated': False,
    'path': 'disk:/test/demofile.txt',
    'public_key': None,
    'public_url': None,
    'type': None,
    'file': None})>

In [ ]:

# Public folder link
public_url = "https://disk.yandex.ru/d/n833udOkQj2ImQ"  # Replace with real link


# Get public resources
resources = y.get_public_resources(public_url=public_url)

In [ ]:
files = y.listdir("https://disk.yandex.ru/d/n833udOkQj2ImQ")

In [ ]:
import os
save_path = '/content/drive/MyDrive/YandexFiles'
os.makedirs(save_path, exist_ok=True)

In [ ]:
# Public folder link
public_url = "https://disk.yandex.ru/d/n833udOkQj2ImQ"  # Replace with real link

# Get public resources
resources = y.get_public_resources(public_url = public_url, public_key=token)

# Step 4: Download files to Google Drive
with open("filelist.txt", "a") as f:
  for item in resources.items:
      if item.type == "file":
          file_path = os.path.join(save_path, item.name)
          f.write(file_path)

with open("filelist.txt") as f:
  print(f.read())

In [ ]:
#Read File List
import requests

# Public folder link (as is)
public_key = "https://disk.yandex.ru/d/n833udOkQj2ImQ"

# Your OAuth token from Yandex
OAUTH_TOKEN = token

headers = {
    "Authorization": f"OAuth {OAUTH_TOKEN}"
}

params = {
    "public_key": public_key,
    "limit": 100  # Adjust as needed; use pagination for more files
}

url = "https://cloud-api.yandex.net/v1/disk/public/resources"

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    items = data['_embedded']['items']
    for item in items:
        if item['type'] == 'dir' and item['name'] == '1. The elephant crossing (outside, inside, clients)':
            print(f"Name: {item['name']} | Type: {item['type']} | Path: {item['path']}")
            file_url = f"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={public_key}&path={item['path']}"
            file_response = requests.get(file_url, headers=headers)
            if file_response.status_code == 200:
                file_data = file_response.json()
                file_data_items = file_data['type'];
                for file_item in file_data_items:
                    if file_item['type'] == 'file':
                        file_download_url = file_item['file']
                        print(f"File Download URL: {file_download_url}")
            else:
                print(f"Error: {file_response.status_code}")
                file_download_url = file_data['href']
                print(f"File Download URL: {file_download_url}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [ ]:
import shutil
from pathlib import Path
import requests



In [ ]:
# Configuration
YANDEX_TOKEN = token
YANDEX_PUBLIC_URL = "https://disk.yandex.ru/d/n833udOkQj2ImQ"
# YANDEX_PUBLIC_URL = "https://disk.yandex.ru/d/n833udOkQj2ImQ/1.%20The%20elephant%20crossing%20(outside%2C%20inside%2C%20clients)"
LOCAL_TEMP_FOLDER = "/content/yandex_temp"
GOOGLE_DRIVE_FOLDER = "/content/drive/MyDrive/YandexPhotosTransfer"

In [ ]:

# Ensure local and drive folders exist
os.makedirs(LOCAL_TEMP_FOLDER, exist_ok=True)
os.makedirs(GOOGLE_DRIVE_FOLDER, exist_ok=True)

In [ ]:

# Initialize Yandex Disk
y = yadisk.YaDisk(token=YANDEX_TOKEN)

In [ ]:
# Configuration
public_key = YANDEX_PUBLIC_URL
base_api_url = "https://cloud-api.yandex.net/v1/disk/public/resources"
headers = {}  # No OAuth needed for public files


In [ ]:
def list_folder(path=""):
    # params = {"public_key": public_key}

    params = {
      "public_key": public_key,
      "limit": 500  # Adjust as needed; use pagination for more files
    }
    if path:
        params["path"] = path
    response = requests.get(base_api_url, headers=headers, params=params)
    return response.json()

In [ ]:
import time
import requests

def download_file_safe(download_url, local_filename, max_retries=3):
    attempt = 0
    while attempt < max_retries:
        try:
            with requests.get(download_url, stream=True, timeout=30) as r:
                if r.status_code == 200:
                    with open(local_filename, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=16384):  # Slightly larger chunks
                            if chunk:  # Filter out keep-alive chunks
                                f.write(chunk)
                    print(f"✅ Downloaded: {local_filename}")
                    return True
                else:
                    print(f"❌ Failed ({r.status_code}): {download_url}")
                    return False
        except Exception as e:
            print(f"⚠️ Error on attempt {attempt+1} for {download_url}: {e}")
            time.sleep(2)
            attempt += 1
    print(f"❌ Failed after retries: {download_url}")
    return False

In [ ]:

def process_folder(path="", parent_drive_path=""):
    data = list_folder(path)
    items = data.get("_embedded", {}).get("items", [])
    for item in items:
        item_path = item["path"]
        item_name = item["name"]
        if item["type"] == "dir":
            new_drive_path = os.path.join(parent_drive_path, item_name)
            new_drive_path
            if not os.path.isdir(new_drive_path):
              print(f"The folder '{new_drive_path}' exists.")
              os.makedirs(new_drive_path, exist_ok=True)
            process_folder(item_path, new_drive_path)
        elif item["type"] == "file":
            download_url = item["file"]
            local_filename = os.path.join("/tmp", item_name)
            download_file_safe(download_url, local_filename)
            gdrive_filename = os.path.join(parent_drive_path, item_name)

            # # Download
            # with requests.get(download_url, stream=True) as r:
            #     with open(local_filename, 'wb') as f:
            #         for chunk in r.iter_content(chunk_size=8192):
            #             f.write(chunk)

            # Copy to Google Drive
            shutil.copy(local_filename, gdrive_filename)
            print(f"📁 Copied to Drive: {gdrive_filename}")

            # Delete local file to free space
            os.remove(local_filename)
            print(f"🗑️ Deleted local copy: {local_filename}")

In [ ]:
# Start the process with:
process_folder(path="", parent_drive_path=GOOGLE_DRIVE_FOLDER)

Streaming output truncated to the last 5000 lines.
🗑️ Deleted local copy: /tmp/@zvandrei-6326.jpg
✅ Downloaded: /tmp/@zvandrei-6327.jpg
📁 Copied to Drive: /content/drive/MyDrive/YandexPhotosTransfer/29. Cocktails 02.03.25/@zvandrei-6327.jpg
🗑️ Deleted local copy: /tmp/@zvandrei-6327.jpg
✅ Downloaded: /tmp/@zvandrei-6328.jpg
📁 Copied to Drive: /content/drive/MyDrive/YandexPhotosTransfer/29. Cocktails 02.03.25/@zvandrei-6328.jpg
🗑️ Deleted local copy: /tmp/@zvandrei-6328.jpg
✅ Downloaded: /tmp/@zvandrei-6329.jpg
📁 Copied to Drive: /content/drive/MyDrive/YandexPhotosTransfer/29. Cocktails 02.03.25/@zvandrei-6329.jpg
🗑️ Deleted local copy: /tmp/@zvandrei-6329.jpg
✅ Downloaded: /tmp/@zvandrei-6330.jpg
📁 Copied to Drive: /content/drive/MyDrive/YandexPhotosTransfer/29. Cocktails 02.03.25/@zvandrei-6330.jpg
🗑️ Deleted local copy: /tmp/@zvandrei-6330.jpg
✅ Downloaded: /tmp/@zvandrei-6331.jpg
📁 Copied to Drive: /content/drive/MyDrive/YandexPhotosTransfer/29. Cocktails 02.03.25/@zvandrei-6331.jpg

In [ ]:
#New Process File
import os
import requests
import time
import shutil

def download_file_safe(download_url, local_filename, max_retries=3):
    attempt = 0
    while attempt < max_retries:
        try:
            with requests.get(download_url, stream=True, timeout=30) as r:
                if r.status_code == 200:
                    with open(local_filename, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=16384):  # safer size
                            if chunk:
                                f.write(chunk)
                    print(f"✅ Downloaded: {local_filename}")
                    return True
                else:
                    print(f"❌ HTTP {r.status_code} - {download_url}")
                    return False
        except Exception as e:
            print(f"⚠️ Attempt {attempt+1} failed for {download_url}: {e}")
            time.sleep(2)
            attempt += 1
    print(f"❌ Final fail: {download_url}")
    return False

def process_file(file_name, download_url, local_folder, gdrive_folder):
    local_filename = os.path.join(local_folder, file_name)
    gdrive_filename = os.path.join(gdrive_folder, file_name)

    # checking file name
    if os.path.exists(gdrive_filename):
        print(f"🟡 Skipped (exists): {gdrive_filename}")
        return

    # Download
    success = download_file_safe(download_url, local_filename)
    if success:
        try:
            shutil.copy(local_filename, gdrive_filename)
            print(f"📂 Copied to Drive: {gdrive_filename}")
            os.remove(local_filename)
        except Exception as e:
            print(f"❌ Copy failed for {file_name}: {e}")
    else:
        print(f"❌ Download failed for: {file_name}")

In [ ]:
!du -hs * /content/drive/MyDrive/YandexPhotosTransfer

4.0K	demofile.txt
71G	drive
0	filelist.txt
55M	sample_data
4.0K	yandex_temp


In [ ]:
import os

def count_files_in_folder(folder_path):
    file_count = 0
    for root, dirs, files in os.walk(folder_path):
        file_count += len(files)
    return file_count

# Example usage:
folder_path = '/content/drive/MyDrive/YandexPhotosTransfer'
total_files = count_files_in_folder(folder_path)
print(f"📁 Total files in Google Drive folder: {total_files}")

📁 Total files in Google Drive folder: 0
